## CSV for gathering responses

In [ ]:
import pandas as pd

# Define column names
columns = ["Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12",
           "issue", "p1", "p2", "p3", "p4", "p5", "p6"]

# Create an empty DataFrame
df = pd.DataFrame(columns=columns)

# Save to CSV file
csv_file_path = "response.csv"  # Change path if needed
df.to_csv(csv_file_path, index=False)

print(f"Empty CSV file '{csv_file_path}' created successfully.")


Empty CSV file 'response.csv' created successfully.


## CSV for Association Matrix (6 x 5)

In [ ]:
import csv

# Define the CSV file name
csv_filename = "am.csv"

# Define the column names
columns = ["parameter", "anxiety", "bipolar", "depression", "normal", "ptsd"]

# Define the rows with values under "parameter" and empty values for the other columns
rows = [
    ["self acceptance", "", "", "", "", ""],
    ["positive relations with others", "", "", "", "", ""],
    ["autonomy", "", "", "", "", ""],
    ["environmental mastery", "", "", "", "", ""],
    ["purpose in life", "", "", "", "", ""],
    ["personal growth", "", "", "", "", ""]
]

# Create and write to the CSV file
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(columns)  # Writing header
    writer.writerows(rows)  # Writing rows

print(f"{csv_filename} has been created successfully!")


am.csv has been created successfully!


# Fill Survey Option

In [1]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.3 MB/s eta 0:00:00


In [2]:
%%writefile test2.py

import streamlit as st
import pandas as pd
import datetime
import os
import numpy as np

# Define file paths in Colab
csv_file_path = "/content/response.csv"
image_path = "/content/intro.png"
am_file_path = "/content/am.csv"

# Display title
st.title("Well-being Survey")

# Display the image
if os.path.exists(image_path):
    st.image(image_path, use_container_width=True)
else:
    st.warning("Intro image not found!")

st.info("If you are not sure, predict your probable mental issue using any one of the 6 options available on the left before filling.")

st.warning(
    "There a total of 12 questions : 2 for each of the 6 paramters from Ryff's Scale of Psychological Wellbeing. The Overall Scores are displayed at the end along with the updated Association Matrix.  \n\n"
    "1 → Strongly Disagree  \n"
    "2 → Disagree  \n"
    "3 → Slightly Disagree  \n"
    "4 → Slightly Agree  \n"
    "5 → Agree  \n"
    "6 → Strongly Agree"
)

# Load existing data or create a new CSV file if it doesn't exist
try:
    df = pd.read_csv(csv_file_path)
except FileNotFoundError:
    df = pd.DataFrame(columns=["Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12",
                               "issue", "p1", "p2", "p3", "p4", "p5", "p6", "Date"])

# Dictionary to store responses
responses = {}

# First question with different options
st.markdown(f"**Q00.** What is Your Predicted Mental Issue?")
responses["issue"] = st.radio("", options=["Anxiety", "Bipolar", "Depression", "Normal", "PTSD"], index=2)

# Display remaining questions with radio buttons (1-5 scale)
questions = [
    " When I look at the story of my life, I am pleased with how things have turned out.",
    " In many ways I feel disappointed about my achievements in life.",
    " People would describe me as a giving person, willing to share my time with others.",
    " Maintaining close relationships has been difficult and frustrating for me.",
    " I have confidence in my own opinions, even if they are different from the way most other people think.",
    " I tend to be influenced by people with strong opinions.",
    " In general, I feel I am in charge of the situation in which I live.",
    " The demands of everyday life often get me down.",
    " Some people wander aimlessly through life, but I am not one of them.",
    " I sometimes feel as if I’ve done all there is to do in life.",
    " For me, life has been a continuous process of learning, changing, and growth.",
    " I gave up trying to make big improvements or changes in my life a long time ago."
]

for i, question in enumerate(questions, start=1):
    st.markdown(f"**Q{i:02}.** {question}")  # Fix: Display full question correctly
    responses[f"Q{i}"] = st.radio("", options=[1, 2, 3, 4, 5, 6], index=2, horizontal=True, key=f"q{i}")
    st.write("\n")  # Add a blank line after each question

# Add current date
responses["Date"] = datetime.datetime.now().strftime("%Y-%m-%d")

# Getting the overall score from the responses
def overall_score(responses):
    scores = {
        "Self Acceptance": responses["Q1"] + abs(7 - responses["Q2"]),
        "Positive Relations with Others": responses["Q3"] + abs(7 - responses["Q4"]),
        "Autonomy": responses["Q5"] + abs(7 - responses["Q6"]),
        "Environmental Mastery": responses["Q7"] + abs(7 - responses["Q8"]),
        "Purpose in Life": responses["Q9"] + abs(7 - responses["Q10"]),
        "Personal Growth": responses["Q11"] + abs(7 - responses["Q12"]),
    }

    p_values = {
        "p1": responses["Q1"] + abs(7 - responses["Q2"]),
        "p2": responses["Q3"] + abs(7 - responses["Q4"]),
        "p3": responses["Q5"] + abs(7 - responses["Q6"]),
        "p4": responses["Q7"] + abs(7 - responses["Q8"]),
        "p5": responses["Q9"] + abs(7 - responses["Q10"]),
        "p6": responses["Q11"] + abs(7 - responses["Q12"])
    }

    # Assign p1 to p6 values
    for key, value in p_values.items():
        responses[key] = value

    descriptions = {
        "Self Acceptance": {
            "High": "Possesses a positive attitude toward the self; acknowledges and accepts multiple aspects of self, including good and bad qualities; feels positive about past life.",
            "Medium": "Generally content with self but sometimes struggles with self-doubt; recognizes strengths but occasionally fixates on weaknesses.",
            "Low": "Feels dissatisfied with self; is disappointed with past life; is troubled about certain personal qualities; wishes to be different."
        },
        "Positive Relations with Others": {
            "High": "Has warm, satisfying, trusting relationships; concerned about the welfare of others; capable of strong empathy, affection, and intimacy.",
            "Medium": "Has meaningful relationships but sometimes struggles with trust or emotional openness; values connections but may not always nurture them deeply.",
            "Low": "Has few close relationships; finds it difficult to be warm, open, and concerned about others; is isolated and frustrated in interpersonal relationships."
        },
        "Autonomy": {
            "High": "Is self-determining and independent; able to resist social pressures; regulates behavior from within and follows personal standards.",
            "Medium": "Balances independence with societal expectations; makes personal choices but occasionally influenced by external opinions.",
            "Low": "Is concerned about the expectations of others; relies on judgments of others; conforms to social pressures."
        },
        "Environmental Mastery": {
            "High": "Has a sense of mastery in managing the environment; makes effective use of opportunities; adapts surroundings to personal needs.",
            "Medium": "Generally manages daily life well but sometimes struggles with external challenges; adapts but may not always feel in control.",
            "Low": "Has difficulty managing everyday affairs; feels unable to change or improve surroundings; lacks sense of control over external world."
        },
        "Purpose in Life": {
            "High": "Has goals and a sense of directedness; finds meaning in present and past life; has aims and objectives for living.",
            "Medium": "Seeks purpose but occasionally feels uncertain; has goals but may struggle with long-term direction or motivation.",
            "Low": "Lacks a sense of meaning; has few goals or aims; does not see purpose in past life; has no outlook or beliefs that give life meaning."
        },
        "Personal Growth": {
            "High": "Feels continuous development; open to new experiences; realizes personal potential; sees self-improvement over time.",
            "Medium": "Has a desire to grow but sometimes feels stuck; enjoys learning but may not actively seek change or self-improvement.",
            "Low": "Feels personal stagnation; lacks a sense of improvement; feels uninterested with life; unable to develop new attitudes or behaviors."
        },
    }

    st.subheader("Overall Scores (Max: 12 for each parameter) and Interpretation :")
    for category, score in scores.items():
        score_level = score // 2  # Dividing score by 2

        # Determine scorer type
        if score_level in [1, 2]:
            level = "Low"
            color = "red"
        elif score_level in [3, 4]:
            level = "Medium"
            color = "orange"
        else:
            level = "High"
            color = "green"

        # Display the result
        st.markdown(f"<span style='color:{color}; font-weight:bold;'>**{category}: {score} ({level} Scorer)**</span>", unsafe_allow_html=True)
        st.info(descriptions[category][level])  # Show description based on category and score level

def update_am_csv(response_csv_path, am_csv_path):
    # Mapping of response.csv columns to am.csv rows
    param_mapping = {
        "p1": "self acceptance",
        "p2": "positive relations with others",
        "p3": "autonomy",
        "p4": "environmental mastery",
        "p5": "purpose in life",
        "p6": "personal growth"
    }

    # Load response.csv
    response_df = pd.read_csv(response_csv_path)

    # Load am.csv
    am_df = pd.read_csv(am_csv_path, index_col=0)

    # Get unique issue types and convert to lowercase
    issue_types = response_df["issue"].unique()
    print(issue_types)

    for issue in issue_types:
        for param, am_row in param_mapping.items():
            # Extract values where issue matches
            param_values = response_df.loc[response_df["issue"] == issue, param]

            if param_values.empty:
                continue  # Skip if no matching rows

            # Divide by 2
            divided_values = param_values // 2
            print(issue)
            print(divided_values)

            # Compute mean value of divided values
            mean_value = divided_values.mean()

            # Ensure final_value is an integer
            final_value = int(round(mean_value))

            # Update am.csv and force integer type
            am_df.loc[am_row, issue.lower()] = final_value

            # Convert entire column to integer type
            am_df[issue.lower()] = am_df[issue.lower()].astype(int)

    # Save the updated am.csv
    am_df.to_csv(am_csv_path)
    st.success("Updated Association Matrix successfully.")

    st.subheader("Updated Association Matrix:")
    am_df = pd.read_csv(am_file_path, index_col=0)
    # Display the dataframe as a table (no scrollbars)
    st.table(am_df)

# Submit button
if st.button("Submit Responses"):
    st.success("Responses Submitted Successfully!")

    # Display overall scores
    overall_score(responses)

    # Convert responses to DataFrame
    new_entry = pd.DataFrame([responses])

    # Append new entry to existing DataFrame
    df = pd.concat([df, new_entry], ignore_index=True)

    # Save updated DataFrame back to CSV
    df.to_csv(csv_file_path, index=False)

    # Show last five responses
    with st.expander("View Last 5 Responses:"):
        last_five = df.tail(5)
        st.dataframe(last_five)

    # Count total respondents for today
    today_date = datetime.datetime.now().strftime("%Y-%m-%d")
    total_today = df[df["Date"] == today_date].shape[0]
    st.write(f"**Total Number of Respondents ({today_date}):** {total_today}")

    update_am_csv(csv_file_path, am_file_path)


Writing test2.py


In [3]:
# Import ngrok
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("2ohUKqk37HcGbvwN0s8Y1E2WNxE_39z1gVF3bYq9vFSEm7Wzq") # Replace YOUR_AUTHTOKEN with your actual authtoken

# Kill any existing ngrok processes
ngrok.kill()

# Start Streamlit with nohup
!nohup streamlit run test2.py &

# Create a public URL with ngrok to access the app
public_url = ngrok.connect(addr='8501')
print(f"Public URL: {public_url}")

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://ca96-34-121-19-183.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
ngrok.kill()

## Testing Updation of Association Matrix

In [ ]:
# import streamlit as st
import pandas as pd
import datetime
import os
import numpy as np

def update_am_csv(response_csv_path, am_csv_path):
    # Mapping of response.csv columns to am.csv rows
    param_mapping = {
        "p1": "self acceptance",
        "p2": "positive relations with others",
        "p3": "autonomy",
        "p4": "environmental mastery",
        "p5": "purpose in life",
        "p6": "personal growth"
    }

    # Load response.csv
    response_df = pd.read_csv(response_csv_path)

    # Load am.csv
    am_df = pd.read_csv(am_csv_path, index_col=0)

    # Get unique issue types and convert to lowercase
    issue_types = response_df["issue"].unique()
    print(issue_types)

    for issue in issue_types:
        for param, am_row in param_mapping.items():
            # Extract values where issue matches
            param_values = response_df.loc[response_df["issue"] == issue, param]

            if param_values.empty:
                continue  # Skip if no matching rows

            # Divide by 2
            divided_values = param_values // 2
            print(issue)
            print(divided_values)

            # Normalize values to be between 1 and 6
            # min_val, max_val = divided_values.min(), divided_values.max()
            # if min_val == max_val:  # Avoid division by zero
            #     normalized_values = np.full_like(divided_values, 3)  # Default to mid-range value
            # else:
            #     normalized_values = 1 + (divided_values - min_val) * (5 / (max_val - min_val))

            # Compute final value (mean rounded to nearest integer)
            # final_value = int(np.round(normalized_values.mean(), 0))

            # Compute mean value of divided values
            mean_value = divided_values.mean()

            # Ensure final_value is an integer
            final_value = int(round(mean_value))

            # Update am.csv and force integer type
            am_df.loc[am_row, issue.lower()] = final_value

            # Convert entire column to integer type
            am_df[issue.lower()] = am_df[issue.lower()].astype(int)

    # Save the updated am.csv
    am_df.to_csv(am_csv_path)
    # st.success("Updated Association Matrix successfully.")

    # st.subheader("Updated Association Matrix:")
    # am_df = pd.read_csv(am_file_path, index_col=0)
    # Display the dataframe as a table (no scrollbars)
    # st.table(am_df)

update_am_csv("/content/response.csv", "/content/am.csv")

am_df = pd.read_csv("/content/am.csv", index_col=0)
am_df

['Normal' 'Depression' 'Anxiety' 'Bipolar' 'PTSD']
Normal
0    6
Name: p1, dtype: int64
Normal
0    6
Name: p2, dtype: int64
Normal
0    5
Name: p3, dtype: int64
Normal
0    6
Name: p4, dtype: int64
Normal
0    6
Name: p5, dtype: int64
Normal
0    6
Name: p6, dtype: int64
Depression
1    1
Name: p1, dtype: int64
Depression
1    2
Name: p2, dtype: int64
Depression
1    2
Name: p3, dtype: int64
Depression
1    1
Name: p4, dtype: int64
Depression
1    1
Name: p5, dtype: int64
Depression
1    1
Name: p6, dtype: int64
Anxiety
2    3
5    3
7    3
Name: p1, dtype: int64
Anxiety
2    3
5    3
7    3
Name: p2, dtype: int64
Anxiety
2    2
5    2
7    2
Name: p3, dtype: int64
Anxiety
2    2
5    2
7    2
Name: p4, dtype: int64
Anxiety
2    4
5    4
7    4
Name: p5, dtype: int64
Anxiety
2    3
5    3
7    3
Name: p6, dtype: int64
Bipolar
3    2
6    2
Name: p1, dtype: int64
Bipolar
3    4
6    4
Name: p2, dtype: int64
Bipolar
3    3
6    3
Name: p3, dtype: int64
Bipolar
3    4
6    3
Name: p4, dt

,anxiety,bipolar,depression,normal,ptsd
parameter,,,,,
self acceptance,3,2,1,6,2
positive relations with others,3,4,2,6,3
autonomy,2,3,2,5,1
environmental mastery,2,4,1,6,4
purpose in life,4,4,1,6,2
personal growth,3,3,1,6,2
